# Smoothing Methods – Refactored

Clean, refactored, and best-practice version.

In [ ]:

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.holtwinters import (
    SimpleExpSmoothing,
    Holt,
    ExponentialSmoothing,
)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 6)

SEASONAL_PERIOD = 12
ACF_LAGS = 25


## Helper Functions

In [ ]:

def train_valid_split(df, split_date, value_col="value"):
    train = df.loc[:split_date].copy()
    valid = df.loc[split_date:].copy()
    return train, valid


def forecast_metrics(y_true, y_pred):
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
    }


def evaluate_forecast(model_fit, train_df, valid_df, value_col="value", title=None):
    forecast = model_fit.forecast(len(valid_df))
    residuals = valid_df[value_col] - forecast

    fig, axes = plt.subplots(2, 2, figsize=(14, 8))

    train_df[value_col].plot(ax=axes[0, 0], label="Train")
    valid_df[value_col].plot(ax=axes[0, 0], label="Validation")
    forecast.plot(ax=axes[0, 0], label="Forecast")
    axes[0, 0].legend()
    axes[0, 0].set_title(title or "Forecast vs Actual")

    residuals.plot(ax=axes[0, 1], title="Residuals")
    plot_acf(residuals, lags=ACF_LAGS, ax=axes[1, 0])
    plot_pacf(residuals, lags=ACF_LAGS, ax=axes[1, 1])

    plt.tight_layout()
    plt.show()

    return forecast, residuals, forecast_metrics(valid_df[value_col], forecast)


## Data Loading

In [ ]:

df = pd.read_csv(
    "data.csv",
    parse_dates=["date"],
    index_col="date",
).sort_index()

df.head()


## Train / Validation Split

In [ ]:

train_df, valid_df = train_valid_split(df, "2017-12-01")


## Simple Exponential Smoothing

In [ ]:

ses_fit = SimpleExpSmoothing(train_df["value"]).fit()

ses_forecast, ses_residuals, ses_metrics = evaluate_forecast(
    ses_fit,
    train_df,
    valid_df,
    title="Simple Exponential Smoothing",
)

ses_metrics


## Holt-Winters Additive

In [ ]:

hw_add_fit = ExponentialSmoothing(
    train_df["value"],
    trend="add",
    seasonal="add",
    seasonal_periods=SEASONAL_PERIOD,
).fit()

hw_add_forecast, hw_add_residuals, hw_add_metrics = evaluate_forecast(
    hw_add_fit,
    train_df,
    valid_df,
    title="Holt-Winters Additive",
)

hw_add_metrics
